In [1]:
# !pip install tensorflow

In [2]:
import numpy as np
import random
import tensorflow as tf
from collections import deque
import tensorflow as tf
import gym

In [3]:
env_name = 'CartPole-v1'
env = gym.make(env_name)


In [4]:
# env.reset()
# for step in range(100):
#     env.render()
#     env.step(env.action_space.sample())
#
# env.close()

In [5]:
num_of_obs = env.observation_space.shape[0]
num_of_act = env.action_space.n

In [6]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(16, activation = 'relu', input_shape=(1, num_of_obs)))
model.add(tf.keras.layers.Dense(32, activation = 'relu'))
model.add(tf.keras.layers.Dense(num_of_act, activation = 'linear'))



In [7]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1, 16)             80        
                                                                 
 dense_1 (Dense)             (None, 1, 32)             544       
                                                                 
 dense_2 (Dense)             (None, 1, 2)              66        
                                                                 
Total params: 690 (2.70 KB)
Trainable params: 690 (2.70 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [8]:
target_model = tf.keras.models.clone_model(model)

In [9]:
EPOCHS = 1000
epsilon = 1.0
epsilon_reduce = 0.0001
learning_rate = 0.001
gamma = 0.95

In [10]:
def egas(model, epsilon, obs):
    if np.random.random() > epsilon:
        prediction = model.predict(obs, verbose=0)
        action = np.argmax(prediction)

    else:
        action = env.action_space.sample()

    return action

In [11]:
replay_buffer = deque(maxlen=20000)
update_target_model = 10

In [12]:
def replay(replay_buffer, batch_size, model, target_model):
    if len(replay_buffer) < batch_size:
        return
    samples = random.sample(replay_buffer, batch_size)
    target_batch = []
    zipped_samples = list(zip(*samples))
    states, actions, rewards, new_states, dones = zipped_samples
    targets = target_model.predict(np.array(states), verbose=0)
    q_values = model.predict(np.array(new_states), verbose=0)

    for i in range(batch_size):
        # print(f"{q_value} : q vlaue")
        q_value = max(q_values[i][0])
        # print(f"{q_value} : q max")
        target = targets[i].copy()

        if dones[i]:
            target[0][actions[i]] = rewards[i]

        else:
            target[0][actions[i]] = rewards[i]+q_value*gamma

        target_batch.append(target)

    model.fit(np.array(states), np.array(target_batch), epochs = 1, verbose=0)


In [13]:
def update_model_handler(update_target_model, model, target_model, epoch):
    # print(epoch)
    if epoch > 0 and update_target_model % epoch == 0:
        target_model.set_weights(model.get_weights())

In [14]:
model.compile(loss='mse', optimizer = (tf.keras.optimizers.Adam(learning_rate=learning_rate)))

In [15]:
env_name = 'CartPole-v1'
env = gym.make(env_name)

In [ ]:
best = 0
for epoch in range(EPOCHS):
    obs = env.reset()
    # print(obs)
    obs = obs[0].reshape([1,4])
    done = False
    points = 0
    while not done:
        action = egas(model, epsilon, obs)
        # print(env.step(action))
        next_obs, reward, done, info, check = env.step(action)
        next_obs = next_obs.reshape([1,4])

        replay_buffer.append((obs, action, reward, next_obs, done))
        obs = next_obs
        points+=1
        replay(replay_buffer, 32, model, target_model)

    epsilon = epsilon - epsilon_reduce
    update_model_handler(update_target_model, model, target_model, epoch)
    if points > best:
        best = points

    if epoch%25 == 0:
        print(f"{epoch} : {points} points : {best} : best")


0 : 20 points : 20 : best


C:\Users\bherw\OneDrive\Desktop\Suraj\Reinforment Learning\gridworld\venv\Lib\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):



25 : 34 points : 57 : best
1/1 [==============================] - 0s 14ms/step
50 : 9 points : 68 : best
1/1 [==============================] - 0s 15ms/step
75 : 16 points : 68 : best
1/1 [==============================] - 0s 15ms/step
100 : 24 points : 68 : best
1/1 [==============================] - 0s 14ms/step
